In [3]:
!python --version

Python 3.10.0


In [4]:
import langchain
langchain.__version__


'0.3.20'

In [5]:
### Initialize langchain agents and tools
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

from dotenv import load_dotenv
load_dotenv()
import os
open_api_key = os.getenv("OPENAI_API_KEY")


In [6]:
!pip install -U duckduckgo-search

   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ---------------------------------------- 3.1/3.1 MB 16.7 MB/s eta 0:00:00


In [40]:
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun
search_tool = Tool(
    name= "DuckDuckGo Search",
    func=DuckDuckGoSearchRun().run,
    description="Use this tool to perform web searches such as latest news, entertainement movies etc using DuckDuckGo"
)
llm = OpenAI(temperature =0.5)




In [7]:
!pip install wikipedia

  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11756 sha256=b4794d00a4f499b9336a493655db2be5b5a1e7bfd2f4d2f00cafb80ae3a08bfe
  Stored in directory: c:\users\suhas\appdata\local\pip\cache\wheels\5e\b6\c5\93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
## Initialize the Wikipedia 
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
wikipedia_tool= Tool(
    name= "Wikipedia Search",
    func=WikipediaAPIWrapper().run,
    description="Use this tool to perform wikipedia searches for information "
)

tools=[wikipedia_tool]


In [10]:
import sqlite3

# Connect to SQLite database (creates example.db if it doesn't exist)
connection = sqlite3.connect("example.db")
cursor = connection.cursor()

# Create the inventory table
cursor.execute("""
CREATE TABLE IF NOT EXISTS inventory (
    id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    stock_quantity INTEGER NOT NULL
)
""")

# Insert sample data
cursor.executemany("""
INSERT INTO inventory (product_name, stock_quantity) VALUES (?, ?)
""", [
    ("Product A", 150),
    ("Product B", 200),
    ("Product C", 300),
    ("Product D", 100),
    ("Product E", 50)
])

# Commit and close connection
connection.commit()
connection.close()

print("Database created and populated with sample data.")

Database created and populated with sample data.


In [24]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

db = SQLDatabase.from_uri("sqlite:///example.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
sqllite_tool = toolkit.get_tools()
tools = sqllite_tool


In [13]:
agent = initialize_agent(tools = tools, 
                         llm=llm,
                         agent="zero-shot-react-description",
                        verbose=True
                        )
query1 = "How many units of Product A are in stock?"
response1 = agent.run(query1)
print("response: ", response1)

C:\Users\suhas\AppData\Local\Temp\ipykernel_42060\1771633576.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools = tools,
C:\Users\suhas\AppData\Local\Temp\ipykernel_42060\1771633576.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response1 = agent.run(query1)




> Entering new AgentExecutor chain...
 We need to find the table that contains information about Product A and its stock levels.
Action: sql_db_list_tables
Action Input: ""
Observation: inventory
Thought: Now that we know the table, we can use sql_db_schema to see its fields and sample rows.
Action: sql_db_schema
Action Input: inventory
Observation: 
CREATE TABLE inventory (
	id INTEGER, 
	product_name TEXT NOT NULL, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from inventory table:
id	product_name	stock_quantity
1	Product A	150
2	Product B	200
3	Product C	300
*/
Thought: We need to use a SQL query to retrieve the stock quantity for Product A.
Action: sql_db_query
Action Input: SELECT stock_quantity FROM inventory WHERE product_name = 'Product A'
Observation: [(150,)]
Thought: We can use sql_db_query_checker to double check our query before executing it.
Action: sql_db_query_checker
Action Input: SELECT stock_quantity FROM inventory WHERE product_name = 'Product

In [18]:
!pip install arxiv

  Using cached arxiv-2.1.3-py3-none-any.whl.metadata (6.1 kB)
  Using cached feedparser-6.0.11-py3-none-any.whl.metadata (2.4 kB)
  Using cached sgmllib3k-1.0.0.tar.gz (5.8 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using cached arxiv-2.1.3-py3-none-any.whl (11 kB)
Using cached feedparser-6.0.11-py3-none-any.whl (81 kB)
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6104 sha256=6377da0424519817053b85f25034a001c0312d3cfe9e55cd4c678dde1b5a610a
  Stored in directory: c:\users\suhas\appdata\local\pip\cache\wheels\f0\69\93\a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [29]:
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain.agents import load_tools
arxiv_tool = Tool(
    name="Arxiv Search",
    func= ArxivQueryRun().run,
    description="Use this tool to search academic papers on arXiv. Provide topics, authors, or keywords."
)





In [47]:
##tools = load_tools(["arxiv_tool", "wikipedia_tool", "search_tool", "sqllite_tool"], llm=llm)
tools = [arxiv_tool, search_tool]

In [48]:
agent = initialize_agent(tools, 
                         llm,
                         agent="zero-shot-react-description",
                        verbose=True,
                        handle_parsing_errors=True
                        )


In [ ]:
##query1 = "Search for research papers on Diffusion Models in 2024."
##query1 = "Find recent papers on Generative AI and what is the latest movie trending in Charlotte USA this week"
query1 = "Who is Sam Altman and What are some of the recent papers on Generative AI"
response1 = agent.run(query1)
print("response: ", response1)



> Entering new AgentExecutor chain...
 I should use DuckDuckGo Search to find information about Sam Altman and the latest traffic news in Charlotte NC
Action: DuckDuckGo Search
Action Input: Sam Altman
Observation: Sam Altman (born April 22, 1985, Chicago, Illinois, U.S.) is an American entrepreneur who was president of the start-up accelerator Y Combinator from 2014 to 2019 and chief executive officer (CEO) of the artificial intelligence (AI) company OpenAI beginning in 2019. He has been compared to tech visionaries, including Steve Jobs and Bill Gates, and is known for his belief that artificial ... Sam Altman is the CEO of OpenAI and a prolific venture investor. In 2005, he dropped out of Stanford to found social mapping company Loopt, which sold in 2012 for $43 million; he used proceeds to ... OpenAI CEO Sam Altman had an eventful 2024, and 2025 is shaping up to be just as big. While the 39-year-old entrepreneur has been a household name in Silicon Valley for years now, the rest 